In [186]:
import json
import requests as req
import config as cfg
import pandas as pd

# API keys stored within config file
app_secret = cfg.foursquare_secret['key']
app_id = cfg.foursquare_id['key']
url_append = f'?client_id={app_id}&client_secret={app_secret}'

# Foursquare

In [181]:
fs_ven_list_url = 'https://api.foursquare.com/v2/venues/search'
params = {'v' : '20200731', 'radius' : '500', 'll':'51.044680,-114.063036'}
fs_ven_list = req.get(fs_ven_list_url+url_append, params = params)
fs_ven_list.status_code

200

In [54]:
fs_ven_list.url

'https://api.foursquare.com/v2/venues/search?client_id=JY32WPFJN4OIBMETWOMXSHPBMGTRBN1ZR4KIYBEZKE4M3XVK&client_secret=CWGLVZETS1TP1X52TNSIKOZVK3QOBNCLT30AIQWZ1JT4LKRB&v=20200731&radius=500&ll=51.044680%2C-114.063036'

In [160]:
fs_venues = pd.json_normalize(fs_ven_list.json()['response']['venues'])
fs_venues.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b2836ecf964a520019124e3,Ruth's Chris Steak House,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1620337875,False,"294, 115 - 9th Avenue SE",2nd Floor of Calgary Tower,51.044224,-114.062804,"[{'label': 'display', 'lat': 51.0442241181576,...",53,T2G 0P5,CA,Calgary,AB,Canada,"[294, 115 - 9th Avenue SE (2nd Floor of Calgar...",NaN
1,4b0586e9f964a520c27422e3,Calgary Tower,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",v-1620337875,False,101 9th Avenue SW,at Centre St.,51.044563,-114.063305,"[{'label': 'display', 'lat': 51.04456309752798...",22,T2P 1J9,CA,Calgary,AB,Canada,"[101 9th Avenue SW (at Centre St.), Calgary AB...",NaN
2,56902984498e604b35bb6f96,Rolls-Royce Alberta,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",v-1620337875,False,112 9 Avenue SW,NaN,51.044657,-114.063171,"[{'label': 'display', 'lat': 51.04465678907416...",9,T2P 0T1,CA,Calgary,AB,Canada,"[112 9 Avenue SW, Calgary AB T2P 0T1, Canada]",NaN
3,559d60eb498e69497044effd,RnR Wellness Spa Le Germaine,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1620337875,False,899 Centre St SW,NaN,51.044630,-114.063074,"[{'label': 'display', 'lat': 51.0446296, 'lng'...",6,T2G 1B8,CA,Calgary,AB,Canada,"[899 Centre St SW, Calgary AB T2G 1B8, Canada]",NaN
4,5c7793e4835c9a003253112e,Calgary Marriott Downtown Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1620337875,False,NaN,NaN,51.044689,-114.062879,"[{'label': 'display', 'lat': 51.044689, 'lng':...",11,T2G 5A6,CA,Calgary,AB,Canada,"[Calgary AB T2G 5A6, Canada]",NaN


In [124]:
fs_venues=fs_venues[['id','name', 'categories', 'location.lat','location.lng', 'location.distance']]
id_list = fs_venues['id']
name_list = fs_venues['name']

In [171]:
fs_review_url = 'https://api.foursquare.com/v2/venues/'
params = {'v' : '20200731'}
rating = []
rating_count =[]
name = []

for i in range(len(id_list)):
    venue_id = id_list[i]
    fs_ven_review = req.get(fs_review_url+venue_id+url_append, params = params)
    try:
        rating.append(pd.json_normalize(fs_ven_review.json()['response']['venue'])['rating'])
        rating_count.append(pd.json_normalize(fs_ven_review.json()['response']['venue'])['ratingSignals'])
        name.append(pd.json_normalize(fs_ven_review.json()['response']['venue'])['name'])      
    except:
        pass

In [189]:
fs_review_url = 'https://api.foursquare.com/v2/venues/'
params = {'v' : '20200731'}
venue_id = '4b0586e9f964a520c27422e3'
fs_ven_review = req.get(fs_review_url+venue_id+url_append, params = params)

In [152]:

a = list(zip(name, rating, rating_count))


b = pd.DataFrame(a,columns = [ 'name', 'rating','rating_count'])

In [174]:
b

,name,rating,rating_count
0,"0 Ruth's Chris Steak House Name: name, dtyp...","0 8.2 Name: rating, dtype: float64","0 31 Name: ratingSignals, dtype: int64"
1,"0 Calgary Tower Name: name, dtype: object","0 7.6 Name: rating, dtype: float64","0 143 Name: ratingSignals, dtype: int64"
2,"0 Sky 360 Name: name, dtype: object","0 7.3 Name: rating, dtype: float64","0 48 Name: ratingSignals, dtype: int64"
3,0 Calgary Marriott Downtown Hotel Name: nam...,"0 7.6 Name: rating, dtype: float64","0 54 Name: ratingSignals, dtype: int64"
4,"0 one18empire Name: name, dtype: object","0 7.6 Name: rating, dtype: float64","0 20 Name: ratingSignals, dtype: int64"
5,"0 Le Germain Hotel Calgary Name: name, dtyp...","0 7.6 Name: rating, dtype: float64","0 52 Name: ratingSignals, dtype: int64"
6,"0 Glenbow Museum Name: name, dtype: object","0 8.5 Name: rating, dtype: float64","0 73 Name: ratingSignals, dtype: int64"


In [164]:
rating

[0    8.2
 Name: rating, dtype: float64,
 0    7.6
 Name: rating, dtype: float64,
 0    7.3
 Name: rating, dtype: float64,
 0    7.6
 Name: rating, dtype: float64,
 0    7.6
 Name: rating, dtype: float64,
 0    7.6
 Name: rating, dtype: float64,
 0    8.5
 Name: rating, dtype: float64]

In [165]:
name

[0    Ruth's Chris Steak House
 Name: name, dtype: object,
 0    Calgary Tower
 Name: name, dtype: object,
 0    Sky 360
 Name: name, dtype: object,
 0    Calgary Marriott Downtown Hotel
 Name: name, dtype: object,
 0    one18empire
 Name: name, dtype: object,
 0    Le Germain Hotel Calgary
 Name: name, dtype: object,
 0    Glenbow Museum
 Name: name, dtype: object]

In [166]:
rating_count

[0    31
 Name: ratingSignals, dtype: int64,
 0    143
 Name: ratingSignals, dtype: int64,
 0    48
 Name: ratingSignals, dtype: int64,
 0    54
 Name: ratingSignals, dtype: int64,
 0    20
 Name: ratingSignals, dtype: int64,
 0    52
 Name: ratingSignals, dtype: int64,
 0    73
 Name: ratingSignals, dtype: int64]

# excel export

In [251]:
fs_venue = pd.read_excel('FS_venue.xls')

In [252]:
fs_venue

,ID,Name,Category,Address,Latitude,Longitude,Distance (m),Postal Code,City,Province,Country,Rating,Reviews
0,4b2836ecf964a520019124e3,Ruth's Chris Steak House,Steakhouse,"294, 115 - 9th Avenue SE",51.044224,-114.062804,53,T2G 0P5,Calgary,AB,Canada,8.2,31
1,4b0586e9f964a520c27422e3,Calgary Tower,Monument / Landmark,101 9th Avenue SW,51.044563,-114.063305,22,T2P 1J9,Calgary,AB,Canada,7.6,143
2,56902984498e604b35bb6f96,Rolls-Royce Alberta,Auto Dealership,112 9 Avenue SW,51.044657,-114.063171,9,T2P 0T1,Calgary,AB,Canada,0.0,0
3,559d60eb498e69497044effd,RnR Wellness Spa Le Germaine,Spa,899 Centre St SW,51.044630,-114.063074,6,T2G 1B8,Calgary,AB,Canada,0.0,0
4,5c7793e4835c9a003253112e,Calgary Marriott Downtown Hotel,Hotel,NaN,51.044689,-114.062879,11,T2G 5A6,Calgary,AB,Canada,0.0,0
5,4b68f194f964a52032942be3,Sky 360,Restaurant,101 9 Ave SW,51.044576,-114.063315,22,NaN,Calgary,AB,Canada,7.3,48
6,4cfe6adb47699eb0c9b91515,H&R Block,Financial or Legal Service,Sears South Centre,51.044600,-114.063000,9,T2J 3-J1,Calgary,AB,Canada,0.0,0
7,4b0586e6f964a520a17322e3,Calgary Marriott Downtown Hotel,Hotel,110 9th Avenue SE,51.044837,-114.062601,35,T2G 5A6,Calgary,AB,Canada,7.6,54
8,52f92dcf11d237a439890841,one18empire,New American Restaurant,820 Centre St S,51.044849,-114.062914,20,T2G 5A6,Calgary,AB,Canada,7.6,20
9,57c576e5498e6c6da2550a41,Calgary Marriott fitness center,Gym / Fitness Center,901 9th Ave,51.044796,-114.062649,30,NaN,Calgary,AB,Canada,0.0,0


# Yelp

In [237]:

y_ven_list=json.load(open('Yelp_venue_list.JSON'))

In [242]:
y_venues = pd.json_normalize(y_ven_list['businesses'])

In [246]:
#export to excel
#y_venues.to_excel(r'D:\LighthouseLabs\w2\mini-project-II\y_venue.xlsx', index = False)

In [249]:
y_venue = pd.read_excel('y_venue.xls')

In [250]:
y_venue

,ID,Name,Reviews,Category 1,Category 2,Category 3,Rating,Distance (m),Latitude,Longitude,Address,City,Postal Code,Country,Province
0,kmOeRAvOL1oR6Z-4Ggdx6w,Charcut Roast House,327,Steakhouses,NaN,NaN,4.0,23.918143,51.044727,-114.063370,101-899 Centre Street S,Calgary,T2G 1B8,Canada,AB
1,SKhV8mF40RcsGrJjkuEwvw,Ten Foot Henry,382,Canadian (New),NaN,NaN,4.5,455.424539,51.041090,-114.065970,1209 1st Street SW,Calgary,T2R 0V3,Canada,AB
2,EWj6tgNVl9x8Z7u9nnqdwQ,Workshop Kitchen + Culture,125,Canadian (New),Venues & Event Spaces,NaN,4.5,312.670662,51.047090,-114.064890,608 1 Street SW,Calgary,T2P 1M6,Canada,AB
3,gLUT5YFfZWUe5JbD-zNGfg,One18 Empire,79,Cocktail Bars,Canadian (New),Whiskey Bars,4.0,35.027125,51.044979,-114.062876,820 Centre Street S,Calgary,T2G 5J2,Canada,AB
4,iAhAnPxgKb6fzxrEcoHtlw,OEB Breakfast,222,Breakfast & Brunch,NaN,NaN,4.5,512.711929,51.048952,-114.065795,222 5 Avenue SW,Calgary,T2P 4K3,Canada,AB
5,YpoIDLyRGhgciExGAHRQcw,Native Tongues Taqueria,357,Mexican,Cocktail Bars,NaN,4.0,482.618511,51.041401,-114.067559,235 12 Avenue SW,Calgary,T2R 1H7,Canada,AB
6,t8QL1WTAlkdGnTPwqEwDIA,Klein Harris,55,Desserts,Canadian (New),Cocktail Bars,4.5,135.814212,51.045780,-114.063690,110 8 Avenue SW,Calgary,T2P 0W5,Canada,AB
7,xiWmyJyTa7o1H6uUNH2dEw,Saltlik,146,Steakhouses,Cocktail Bars,Venues & Event Spaces,3.5,71.983812,51.045430,-114.063290,101 8 Avenue SW,Calgary,T2P 1B4,Canada,AB
8,bFLEbPKUXmeQcBciGdHjww,Veg-In YYC,75,Vegan,Vegetarian,Indian,4.5,391.706074,51.047194,-114.059111,215 6th Avenue SE,Calgary,T2G 0G4,Canada,AB
9,8dnH_PcGlzECcCGk_5yyww,Briggs Kitchen + Bar,204,Cocktail Bars,Sandwiches,Burgers,4.0,463.436949,51.043278,-114.069279,100 - 317 10th Avenue SW,Calgary,T2R 0A5,Canada,AB
